# Library

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import FastText
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from keras.utils.np_utils import to_categorical
from keras.models import load_model, Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Preprocessing
### Load Data

In [2]:
df = pd.read_pickle('preprocessed_df.pkl')
df = pd.DataFrame(df[['detokenize','plot']])
df.head()

,detokenize,plot
0,the desperate hour lakewood salah cerita suara...,1
1,edisi males review singkat tonton libur dp des...,1
2,plot utama orang deserter pursuit buru wamil j...,1
3,film hereditary horror thrill midsommar gatau ...,0
4,batman manusiawi tarung nya sadis scene pursui...,1


In [3]:
reviews = df['detokenize']
label = df['plot']

# Variables

In [4]:
MAX_FEATURES = 10000
test_size_split = 0.1

# Feature Extraction

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features = MAX_FEATURES)

tfidf = vectorizer.fit_transform(reviews.astype(str))

# SMOTE

In [6]:
smote = SMOTE()

x_smote, y_smote = smote.fit_resample(tfidf, label)

# Split Data

In [7]:
x_train, x_test, y_train, y_test = (
    train_test_split(x_smote, 
                     y_smote, 
                     test_size=test_size_split, 
                     stratify = y_smote, 
                     random_state=42)
)

print(f'Train\t| X shape: {x_train.shape}\tY shape: {y_train.shape}')
print(f'Test\t| X shape: {x_test.shape}\tY shape: {y_test.shape}')

Train	| X shape: (18897, 10000)	Y shape: (18897,)
Test	| X shape: (2100, 10000)	Y shape: (2100,)


# Feature Expansion

In [8]:
# Load trained fasttext model
model_exp = FastText.load('fasttext_news_tweet.model').wv

In [9]:
def feature_expansion(df, feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 1) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [10]:
feature_ext = vectorizer.get_feature_names_out()

df_x_train = pd.DataFrame(x_train.todense(), columns = feature_ext)
df_x_test = pd.DataFrame(x_test.todense(), columns = feature_ext)

df_x_train_exp = feature_expansion(df_x_train, feature_ext)
df_x_test_exp = feature_expansion(df_x_test, feature_ext)

x_train = np.expand_dims(df_x_train_exp, 1)
x_test = np.expand_dims(df_x_test_exp, 1)

y_train = pd.get_dummies(y_train).values
y_test = pd.get_dummies(y_test).values

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:51<00:00, 89.89it/s]


# Model RNN

In [11]:
def RNN(x_train_c, y_train_c, x_test_c, y_test_c):
    rnn = Sequential()
    rnn.add(SimpleRNN(units=256, activation='relu', recurrent_dropout=0.2, return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=128, activation='relu', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='relu'))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=3, activation='softmax'))
    
    rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    rnn.build(x_train_c.shape)
    
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', verbose = 1, patience=3, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint('rnn_tfidf_ft_smote_plot.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
    
    history = rnn.fit(x_train_c, y_train_c, epochs= 10, batch_size=128, 
                      validation_data=(x_test_c, y_test_c), verbose = 1,
                      callbacks=[early_stopping, model_checkpoint])
    
    model = load_model('rnn_tfidf_ft_smote_plot.h5')
    y_pred = model.predict(x_test_c)
    y_pred_cat = y_pred.argmax(axis=1)
    y_test_cat = np.argmax(y_test_c, axis=1)
   
    cm = confusion_matrix(y_test_cat, y_pred_cat)
    classreport = classification_report(y_test_cat, y_pred_cat)
    f1 = f1_score(y_test_cat, y_pred_cat,  average="macro")
    recall = recall_score(y_test_cat, y_pred_cat, average="macro")
    precision = precision_score(y_test_cat, y_pred_cat, average="macro")
    accuracy = accuracy_score(y_test_cat, y_pred_cat)

    print(classreport)
    print("F1 Score : ", f1)
    print("Precision : ", precision)
    print("Recall : ", recall)
    print("Accuracy : ", accuracy)

    return [f1, precision, recall, accuracy, cm]

# Running

In [12]:
hasil = []

for i in range(1, 6):
    hasil.append([str(i)] + RNN(x_train, y_train, x_test, y_test))

Epoch 1/10
148/148 [==============================] - ETA: 0s - loss: 0.7950 - accuracy: 0.6494
Epoch 1: val_accuracy improved from -inf to 0.72286, saving model to rnn_tfidf_ft_smote_plot.h5
148/148 [==============================] - 14s 44ms/step - loss: 0.7950 - accuracy: 0.6494 - val_loss: 0.6578 - val_accuracy: 0.7229
Epoch 2/10
147/148 [============================>.] - ETA: 0s - loss: 0.4926 - accuracy: 0.8114
Epoch 2: val_accuracy improved from 0.72286 to 0.75095, saving model to rnn_tfidf_ft_smote_plot.h5
148/148 [==============================] - 5s 31ms/step - loss: 0.4929 - accuracy: 0.8113 - val_loss: 0.6292 - val_accuracy: 0.7510
Epoch 3/10
147/148 [============================>.] - ETA: 0s - loss: 0.3075 - accuracy: 0.8890
Epoch 3: val_accuracy improved from 0.75095 to 0.76000, saving model to rnn_tfidf_ft_smote_plot.h5
148/148 [==============================] - 5s 31ms/step - loss: 0.3074 - accuracy: 0.8890 - val_loss: 0.6838 - val_accuracy: 0.7600
Epoch 4/10
148/148 [=

66/66 [==============================] - 2s 8ms/step
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       700
           1       0.76      0.74      0.75       700
           2       0.74      0.71      0.72       700

    accuracy                           0.77      2100
   macro avg       0.77      0.77      0.77      2100
weighted avg       0.77      0.77      0.77      2100

F1 Score :  0.772807826573219
Precision :  0.7725137994914872
Recall :  0.7742857142857144
Accuracy :  0.7742857142857142
Epoch 1/10
147/148 [============================>.] - ETA: 0s - loss: 0.7927 - accuracy: 0.6549
Epoch 1: val_accuracy improved from -inf to 0.73762, saving model to rnn_tfidf_ft_smote_plot.h5
148/148 [==============================] - 16s 46ms/step - loss: 0.7923 - accuracy: 0.6552 - val_loss: 0.6475 - val_accuracy: 0.7376
Epoch 2/10
148/148 [==============================] - ETA: 0s - loss: 0.4965 - accuracy: 0.8086
Epoch 2: val_accuracy i

# Save Output

In [13]:
df = pd.DataFrame(hasil, columns =['iterasi', 'f1', 'precision', 'recall', 'accuracy', 'cm'])
#df.to_excel('rnn_tfidf_ft_smote_plot.xlsx')
df.head()

,iterasi,f1,precision,recall,accuracy,cm
0,1,0.766459,0.766196,0.767619,0.767619,"[[603, 41, 56], [58, 526, 116], [75, 142, 483]]"
1,2,0.774021,0.774261,0.773810,0.773810,"[[589, 47, 64], [52, 521, 127], [51, 134, 515]]"
2,3,0.769169,0.769337,0.769048,0.769048,"[[594, 52, 54], [45, 518, 137], [56, 141, 503]]"
3,4,0.772808,0.772514,0.774286,0.774286,"[[616, 37, 47], [59, 516, 125], [79, 127, 494]]"
4,5,0.777231,0.778166,0.777143,0.777143,"[[598, 35, 67], [50, 505, 145], [52, 119, 529]]"


In [14]:
print("Accuracy : %.2f" % (df['accuracy'].mean()*100))
print("F1-Score : %.2f" % (df['f1'].mean()*100))

Accuracy : 77.24
F1-Score : 77.19
